# Automatinio įrankio naudojimo pavyzdys


## Importuokite Reikalingus Paketus


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Kliento kūrimas

Šiame pavyzdyje naudosime [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) prieigai prie LLM.

`modelis` apibrėžtas kaip `gpt-4o-mini`. Pabandykite pakeisti modelį į kitą, esantį GitHub Models rinkoje, kad pamatytumėte skirtingus rezultatus.

Kaip greitą testą, tiesiog paleisime paprastą užklausą - `Kokia yra Prancūzijos sostinė`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Funkcijų apibrėžimas

Šiame pavyzdyje suteiksime agentui prieigą prie įrankio, kuris yra funkcija su galimų atostogų krypčių sąrašu ir jų prieinamumu.

Galite įsivaizduoti, kad tai būtų situacija, kai kelionių agentas, pavyzdžiui, turi prieigą prie kelionių duomenų bazės.

Peržiūrėdami šį pavyzdį, drąsiai bandykite apibrėžti naujas funkcijas ir įrankius, kuriuos agentas galėtų naudoti.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Funkcijos įrankio apibrėžimas
Kad agentas galėtų naudoti `vacation_destinations` kaip `FunctionTool`, turime jį taip apibrėžti.

Taip pat pateiksime įrankio aprašymą, kuris padės agentui suprasti, kam šis įrankis yra skirtas atsižvelgiant į vartotojo pateiktą užduotį.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Agento apibrėžimas

Dabar galime sukurti agentą naudodami žemiau pateiktą kodą. Mes apibrėžiame `system_message`, kad suteiktume agentui instrukcijas, kaip padėti vartotojams rasti atostogų vietas.

Taip pat nustatome `reflect_on_tool_use` parametrą į true. Tai leidžia naudoti LLM, kad būtų galima paimti įrankio iškvietimo atsakymą ir pateikti jį natūralia kalba.

Galite nustatyti šį parametrą į false, kad pamatytumėte skirtumą.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Agento paleidimas

Dabar galime paleisti agentą su pradiniu vartotojo pranešimu, kuriame prašoma suplanuoti kelionę į Tokiją.

Galite pakeisti šį miesto tikslą, kad pamatytumėte, kaip agentas reaguoja į miesto prieinamumą.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
